In [3]:
import requests
import json
import sqlite3
import time
import folium
import pandas as pd
from plotly import express as px

#create our own database
conn = sqlite3.connect('traffic_data.db')

def insert_data(conn, data):
    """Inserts new traffic data into incidents table in database"""

    if len(data) > 0:
        data.to_sql("incidents", conn, if_exists="append", index=False)

def get_traffic_data(bbox, key):
    """retrieves traffic incident data in a given bounding box from MapQuest Traffic API"""

    # key = ""
    response = requests.get(f"https://www.mapquestapi.com/traffic/v2/incidents?key={key}&boundingBox={bbox}&filters=congestion,incidents,construction,event")
    data = pd.DataFrame(response.json()["incidents"])
    if len(data) > 0:
        data = data[['id', 'type', 'severity', 'shortDesc', 'lat', 'lng']]
    return data

def store_traffic_data(conn, lat_start, lat_end, lng_start, lng_end):
    """Iterates over a given area and updates traffic incident database by MapQuest API calls"""

    bbox_step = 1  # Bbox step size for iterating over the U.S. 
    bbox_range = {
        # Starting latitude for bbox
        "lat_start": lat_start, # 24.396308,  #southernmost
        # Ending latitude for bbox
        "lat_end": lat_end, # 49.384358,    #northernmost
        # Starting longitude for bbox
        "lng_start": lng_start, # -125.000000,  #westernmost
        # Ending longitude for bbox
        "lng_end": lng_end # -66.934570    #easternmost
    }
    # create_table()
    bbox = {
        "lat_start": bbox_range["lat_start"],
        "lat_end": bbox_range["lat_start"] + bbox_step,
        "lng_start": bbox_range["lng_start"],
        "lng_end": bbox_range["lng_start"] + bbox_step
    }
    page = 1
    # a loop that continues until the latitude of the current 
    #bounding box exceeds the northernmost latitude of the bbox_range.
    while bbox["lat_start"] <= bbox_range["lat_end"]:
        #to fetch traffic incident data for the current bounding box
        data = get_traffic_data(f"{bbox['lat_start']},{bbox['lng_start']},{bbox['lat_end']},{bbox['lng_end']}")
        insert_data(conn, data)
        print(f"Page {page} processed.")
        page += 1
        #the longitude values of the bounding box are updated by adding the bbox_step 
        #value to both the starting and ending longitudes
        bbox["lng_start"] += bbox_step
        bbox["lng_end"] += bbox_step
        #If the updated longitude exceeds the easternmost longitude of the bbox_range, 
        #the latitude values are updated, and the longitude values are reset to the starting values
        if bbox["lng_start"] > bbox_range["lng_end"]:
            bbox["lat_start"] += bbox_step
            bbox["lat_end"] += bbox_step
            bbox["lng_start"] = bbox_range["lng_start"]
            bbox["lng_end"] = bbox_range["lng_start"] + bbox_step
        time.sleep(1)  # Sleep for 1 second to avoid hitting API rate limits

def get_incidents_in_area(conn, bbox):
    """Retrieves incidents within the given bounding box from database"""
    
    min_lat, max_lat = bbox[0], bbox[1]
    min_lng, max_lng = bbox[2], bbox[3]
    cmd=\
        f"""
            SELECT * FROM incidents
            WHERE lat BETWEEN {min_lat} AND {max_lat}
            AND lng BETWEEN {min_lng} AND {max_lng}
        """
    print(cmd)
    df = pd.read_sql_query(cmd, conn)
    return df

def display_map_with_incidents(incidents, **kwargs):
    """Creates a plotly map of traffic incidents"""
    
    # try:
    #     center_lat, center_lng = incidents['lat'][0], incidents['lng'][0]
    # except:
    #     print("data frame error?")
    #     center_lat, center_lng = 0, 0

    fig = px.scatter_mapbox(incidents,
                            lat="lat",
                            lon="lng",
                            color="severity",
                            hover_name="id",
                            hover_data=['shortDesc', 'type'],
                            mapbox_style="open-street-map",
                            **kwargs)
    fig.update_layout(margin={"r":0, "l":0,"b":0,"t":0})
    return fig


In [1]:
import sqlite3
import pandas as pd

In [3]:
conn.close()

NameError: name 'conn' is not defined

In [10]:
data=pd.DataFrame({
    'book_id':[12345,12346,12347,12348],
    'title':['Python Programming','Learn MySQL','Data Science Cookbook','bruh'],
    'price':[29,23,27,30]
})
data

,book_id,title,price
0,12345,Python Programming,29
1,12346,Learn MySQL,23
2,12347,Data Science Cookbook,27
3,12348,bruh,30


In [ ]:
import credentials as cred
import requests
key=cred.mapquest_api_key
bbox="39.95,-105.25,39.52,-104.71"
bbox="40,-122,39,-121"
response = requests.get(f"https://www.mapquestapi.com/traffic/v2/incidents?key={key}&boundingBox={bbox}&filters=construction,incidents")
response.json().keys()
df = pd.DataFrame(response.json()["incidents"])

In [40]:
df = df[['id', 'type', 'severity', 'shortDesc', 'lat', 'lng', 'startTime', 'endTime']]

In [42]:
from datetime import datetime
from datetime import timedelta

tdate = '2023-05-22T16:01:34'
sdate = '2023-05-09T15:01:34'

In [56]:
df.to_sql('testdate', conn, if_exists='append')

19

In [65]:
time.strftime('%Y-%m-%d %H:%M:%S')

'2023-05-23 11:03:03'

In [69]:
time = datetime.now() + timedelta(hours=12)
cmd=\
f"""
    SELECT * FROM testdate
    WHERE endDatetime < '{time.strftime('%Y-%m-%d %H:%M:%S')}'
"""
df = pd.read_sql_query(cmd, conn)
len(df)

5

In [59]:
df['endDatetime'] = df['endTime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))
test = df[df['endDatetime'] - timedelta(hours=12) < datetime.now()]
len(test)

5

In [35]:
dtobj1 = datetime.strptime(tdate,"%Y-%m-%dT%H:%M:%S")
dtobj2 = datetime.strptime(sdate,"%Y-%m-%dT%H:%M:%S")

In [36]:
now = datetime.now()
dtobj1 + timedelta(hours=5) < now

True

In [8]:
data['first'] = data['title'].apply(lambda x: str.split(x)[0])

In [29]:
data

,book_id,title,price
0,12345,Python Programming,29
1,12346,Learn MySQL,23
2,12347,Data Science Cookbook,27
3,12348,bruh,30


In [8]:
conn = sqlite3.connect('test.db')

In [30]:
data.to_sql('testtable',conn,if_exists='append')

4

In [31]:
data.to_sql('testtable2',conn,if_exists='append')

4

In [3]:
cmd = "SHOW TABLES LIKE 'testtable'"
cursor = conn.cursor()
cursor.execute(cmd)
cursor.fetchone()

OperationalError: near "SHOW": syntax error

In [37]:
data.to_sql('testtable2',conn,if_exists='append')

4

In [10]:
# data = data.set_index('book_id')
data

,title,price
book_id,,
12345,Python Programming,29
12346,Learn MySQL,23
12347,Data Science Cookbook,27
12348,bruh,30


In [35]:
cursor = conn.cursor()
cmd=\
"""
DELETE FROM testtable2
WHERE book_id IN (SELECT book_id FROM testtable)
"""
cursor.execute(cmd)

In [39]:
cursor = conn.cursor()
cmd=\
"""
INSERT INTO testtable
SELECT * from testtable2
"""
cursor.execute(cmd)

In [40]:
cmd=\
"""
    SELECT * from testtable
"""
df = pd.read_sql_query(cmd, conn)
df

,index,book_id,title,price
0,0,12345,Python Programming,29
1,1,12346,Learn MySQL,23
2,2,12347,Data Science Cookbook,27
3,0,12345,Python Programming,29
4,1,12346,Learn MySQL,23
5,2,12347,Data Science Cookbook,27
6,3,12348,bruh,30
7,0,12345,Python Programming,29
8,1,12346,Learn MySQL,23
9,2,12347,Data Science Cookbook,27


In [36]:
cmd=\
"""
    SELECT * FROM testtable t1
    FULL OUTER JOIN testtable2 t2
    ON t1.book_id=t2.book_id
    WHERE t1.book_id IS NULL
    OR t2.book_id IS NULL
"""
cmd=\
"""
    SELECT * FROM testtable t1
    UNION
    SELECT * FROM testtable2 t2
"""
df = pd.read_sql_query(cmd, conn)
df

,index,book_id,title,price
0,0,12345,Python Programming,29
1,1,12346,Learn MySQL,23
2,2,12347,Data Science Cookbook,27
3,3,12348,bruh,30
